# Setup

In [1]:
import os
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt

# Keras
#from keras.models import Sequential
#from keras.layers import Dense, Flatten

In [2]:
DATASET_PATH = os.path.join(os.getcwd(), 'dataset')

In [3]:
plt.switch_backend('TkAgg')
rsEEG_pre = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, "TMS-EEG-H_02_S1b_rsEEG_pre.vhdr"), preload=True)
#rsEEG_post = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, "TMS-EEG-H_02_S1b_rsEEG_post.vhdr"), preload=True)
#spTEP_pre = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, "TMS-EEG-H_02_S1b_spTEP_pre.vhdr"), preload=True)
#spTEP_post = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, "TMS-EEG-H_02_S1b_spTEP_post.vhdr"), preload=True)

Extracting parameters from /home/tomasgalle/UGent/thesis/tms-research/dataset/TMS-EEG-H_02_S1b_rsEEG_pre.vhdr...
Setting channel info structure...
Reading 0 ... 3984899  =      0.000 ...   796.980 secs...


In [48]:
rsEEG_pre.info # Same info for all files

Measurement date,"August 23, 2017 15:56:21 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,67 points
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,5000.00 Hz
Highpass,0.00 Hz
Lowpass,1000.00 Hz


# EDA

In [67]:
info = rsEEG_pre.info
ch1 = "TP9"
ch2 = "HEOG"

ch1_index = mne.pick_channels(info['ch_names'], include=[ch1])
ch2_index = mne.pick_channels(info['ch_names'], include=[ch2])
ch1_coords = info['chs'][ch1_index[0]]['loc'][:3]
ch2_coords = info['chs'][ch2_index[0]]['loc'][:3]
distance = np.linalg.norm(np.array(ch1_coords) - np.array(ch2_coords))

print(f"Distance between channels: {distance * 100:.3f} cm")

Distance between channels: 0.226 cm


In [56]:
# Sensor analysis
print(f"(BEFORE) Found {len(rsEEG_pre.ch_names)} sensors")
print("(BEFORE) List of sensors:", rsEEG_pre.ch_names)

rsEEG_pre.plot_sensors(show_names=True, kind="3d")
# Channels Fp2 and VEOG overlap and cause issues later on, so we drop VEOG
if("VEOG" in rsEEG_pre.ch_names):
   rsEEG_pre = rsEEG_pre.drop_channels(["VEOG", "VEOG"])
rsEEG_pre.plot_sensors(show_names=True, kind="3d")

print(f"(AFTER) Found {len(rsEEG_pre.ch_names)} sensors")
print("(AFTER) List of sensors:", rsEEG_pre.ch_names)

(BEFORE) Found 64 sensors
(BEFORE) List of sensors: ['Iz', 'O2', 'Oz', 'O1', 'PO8', 'PO4', 'POz', 'PO3', 'PO7', 'P8', 'P6', 'P4', 'P2', 'Pz', 'P1', 'P3', 'P5', 'P7', 'TP10', 'TP8', 'CP6', 'CP4', 'CP2', 'CPz', 'CP1', 'CP3', 'CP5', 'TP7', 'TP9', 'T8', 'C6', 'C4', 'C2', 'Cz', 'C1', 'C3', 'C5', 'T7', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'FC1', 'FC3', 'FC5', 'FT7', 'F8', 'F6', 'F4', 'F2', 'Fz', 'F1', 'F3', 'F5', 'F7', 'AF4', 'AFz', 'AF3', 'Fp2', 'Fpz', 'Fp1', 'VEOG', 'HEOG']
(AFTER) Found 63 sensors
(AFTER) List of sensors: ['Iz', 'O2', 'Oz', 'O1', 'PO8', 'PO4', 'POz', 'PO3', 'PO7', 'P8', 'P6', 'P4', 'P2', 'Pz', 'P1', 'P3', 'P5', 'P7', 'TP10', 'TP8', 'CP6', 'CP4', 'CP2', 'CPz', 'CP1', 'CP3', 'CP5', 'TP7', 'TP9', 'T8', 'C6', 'C4', 'C2', 'Cz', 'C1', 'C3', 'C5', 'T7', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'FC1', 'FC3', 'FC5', 'FT7', 'F8', 'F6', 'F4', 'F2', 'Fz', 'F1', 'F3', 'F5', 'F7', 'AF4', 'AFz', 'AF3', 'Fp2', 'Fpz', 'Fp1', 'HEOG']


In [4]:
# EEG analysis
rsEEG_pre.plot(duration=10, n_channels=20, scalings={'eeg': 75e-6})

# Mark visually identified channels as bad
# rsEEG_pre.info['bads'] = ['MEG 2443']

# spectrum_rsEEG_pre = rsEEG_pre.compute_psd()
# spectrum_rsEEG_pre.plot()
# spectrum_rsEEG_pre.plot_topomap()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 1920x1136 with 4 Axes>

Channels marked as bad:
none


In [46]:
# spTEP even analysis
#events_spTEP_pre, _ = mne.events_from_annotations(spTEP_pre)
#events_spTEP_post, _ = mne.events_from_annotations(spTEP_post)

# df = pd.DataFrame(events_spTEP_pre, columns=['time', 'duration', 'event type'])
# df = df.drop(df.index[0])
# df = df.drop(columns=['duration', 'event type'])

# events_spTEP_pre
# TODO: What do the values mean?

In [45]:
# epochs_spTEP_pre = mne.Epochs(spTEP_pre, events_spTEP_pre, 1, tmin=0, tmax=1, baseline=None, preload=True)
# #print(epochs.get_data())
# #print("Shape of the 3D array (trials, channels, time points):", epochs.get_data().shape)

# p180_idx = epochs_spTEP_pre.time_as_index(0.18)
# p180_data = epochs_spTEP_pre.get_data()[:, :, p180_idx]
# # average_value_at_180ms = data_at_180ms.mean(axis=0)
# # positive_matrix = average_value_at_180ms > 0

# print(np.mean(p180_data))

# EEG preprocessing
EEG preprocessing is potentially not needed for higher performance, as shown in [this paper](https://iopscience.iop.org/article/10.1088/1741-2552/ab0ab5/pdf), but can still be useful.

# Dataset augmentation
Because the dataset is really small, it's highly necessary to do some data augmentation to grow the dataset to be able to train the models on.

# Basic model test
This model trains on false data (every pulse), but currently it's here to test how to train a model on EEG data in general.

In [ ]:
# Prepare data and labels
X = epochs.get_data()  # EEG data
y = (events[:, 2] == 1).astype(int)  # Binary labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

# Define the Keras model
model = Sequential()
model.add(Flatten(input_shape=(X_train.shape[1], X_train.shape[2])))  # Flatten the input
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))